In [1]:
import numpy as np
import pandas as pd
import os, datetime
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
print('Tensorflow version: {}'.format(tf.__version__))

import matplotlib.pyplot as plt
plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

Tensorflow version: 2.4.1


In [2]:
zz_800 = pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/is_zz800.csv',index_col =0)
zz_800 = zz_800['2013':'2021-07-18']

In [3]:
open_price = pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/open_price_post.csv',index_col =0)
open_price = open_price['2013':'2021-07-18']

In [4]:
close_price = pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/close_price_post.csv',index_col =0)
close_price = close_price['2013':'2021-07-18']

In [5]:
rtn_1 =close_price/close_price.shift(1)-1
rtn_20 = close_price/close_price.shift(20)-1
rtn_60 = close_price/close_price.shift(60)-1
rtn_120 = close_price/close_price.shift(120)-1
rtn_250 = close_price/close_price.shift(250)-1

In [6]:
p_rtns = rtn_1.unstack().reset_index()
p_rtns.columns =['stock','dates','rtn_1']

In [7]:
comp_dict={
    'rtn_20':rtn_20,
    'rtn_60':rtn_60,
    'rtn_120':rtn_120,
    'rtn_250':rtn_250
}

In [ ]:
for df_name,df in comp_dict.items():
    df = df.unstack().reset_index()
    df.columns =['stock','dates',df_name]
    p_rtns =pd.merge(p_rtns, df,how ='left',on = ['stock','dates'])

In [ ]:
idx =[]
for name, row in rtns.iterrows():
#     print(name)
#     print(row.isnull().all())
    if row.isnull().all()==True:
        idx.append(name)

In [ ]:
p_rtns.drop(p_rtns.index.isin(idx))

In [ ]:
p_rtns

In [ ]:
# tempp = f_df.reset_index()
def ft_rtn(stock,q):
    dates = np.array(new[new['stock']==stock]['dates'].values)
#     dates = [i if (i in dates)& (i<'2021-07-12')]
    op = price[stock]
    rtn_f = pd.concat([calc_rtn(20,dates,op),calc_rtn(60,dates,op),calc_rtn(120,dates,op),calc_rtn(250,dates,op),calc_rtn(1,dates,op)],axis=1)
    rtn_f.columns =['rtn_20','rtn_60','rtn_120','rtn_250','rtn_1']
    rtn_f['dates']=dates
    rtn_f['stock'] = [stock]*len(rtn_f)
    return q.put({stock:rtn_f})
#     tot.append(rtn_f)

def calc_rtn(days,dates,op):
    tmr =[list(price.index).index(i)+1 for i in dates]
#     tmr = [i if (i in tmr)&(i<2071) for i in tmr]
    ftr = [list(price.index).index(i)+1+days for i in dates]
    ftr = [i if (i in ftr)&(i<2075) else 2074 for i in ftr]
    return pd.Series(op.iloc[ftr].values/op.iloc[tmr].values-1)

q = multiprocessing.Manager().Queue()
param = []
for stock in set(new['stock']):
    param.append((stock,q))
# print(param)
#task
with multiprocessing.Pool(processes =10) as pool:
    pool.starmap(ft_rtn,param)

#extract
elements = []
while q.qsize():
    elements.append(q.get())

elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = pd.DataFrame(elements[i][stock])
    elmts.append(exp)

In [8]:
quant=pd.read_csv('/home/hengtian/kate/signal/overall_signals_v4.csv',index_col = 0)
quant.columns

Index(['stock', 'dates', 'npc_y1', 'npc_rank', 'npc_y2', 'npcy2_rank', 'rate',
       'rate_rank', 'moat', 'moat_rank', 'np_g1', 'np_g1_rank', 'lr',
       'lr_rank', 'growth_cur', 'growth_cur_rank', 'growth_prev',
       'growth_prev_rank', 'interval_days', 'n_pct', 'max_pct',
       'close_vs_forty', 'high_vs_20', 'amt_past5', 'tov_60',
       'max700_vs_min100', 'min_12_100', 'time_f_ipo', 'days_btw_max',
       'max_700', 'min_100', 'close_price', 'buy_price', 'zz_max_vs_min_20',
       'zz_max700_min100', 'rtn_1', 'alpha_1'],
      dtype='object')

In [9]:
quant = quant[['stock','dates','n_pct', 'max_pct','close_vs_forty', 'high_vs_20', 'amt_past5', 'tov_60','max700_vs_min100', 'min_12_100', 'time_f_ipo', 'days_btw_max','max_700', 'min_100', 'close_price', 'buy_price', 'zz_max_vs_min_20','zz_max700_min100', 'rtn_1', 'alpha_1']]

In [14]:
len(quant.columns)

20

In [10]:
from numpy.lib.stride_tricks import as_strided as stride

In [11]:
array=[]
v = quant.values
d0, d1 = v.shape
s0, s1 = v.strides
w = 20
a = stride(v, (d0 - (w - 1), w, d1), (s0, s0, s1))

for row, values in zip(quant.index, a):
    row = pd.DataFrame(values, columns=quant.columns)
#     print(row)
    array.append(row)
#     return rolled_df.groupby(level=0, **kwargs)

In [13]:
print(np.array(array).shape)

(2883753, 20, 20)


In [ ]:
def roll(df, w, **kwargs):
    v = df.values
    d0, d1 = v.shape
    s0, s1 = v.strides

    a = stride(v, (d0 - (w - 1), w, d1), (s0, s0, s1))

    rolled_df = ({
        row: pd.DataFrame(values, columns=df.columns)
        for row, values in zip(df.index, a)
    })

    return rolled_df
np.array(roll(quant,20))

In [18]:
y_1 = quant[['stock','dates','rtn_1']]
table_y =  pd.pivot_table(y_1,index='dates',columns='stock')

rtn_1                                                  \
stock      000001.XSHE 000002.XSHE 000004.XSHE 000005.XSHE 000006.XSHE   
dates                                                                    
2014-07-03         NaN         NaN         NaN         NaN         NaN   
2014-07-04         NaN         NaN         NaN         NaN         NaN   
2014-07-07         NaN         NaN         NaN         NaN         NaN   
2014-07-08         NaN         NaN         NaN         NaN         NaN   
2014-07-09         NaN         NaN         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2021-07-07         NaN         NaN    0.040216         NaN         NaN   
2021-07-08         NaN         NaN   -0.013585         NaN         NaN   
2021-07-09         NaN         NaN    0.199155         NaN         NaN   
2021-07-12         NaN         NaN   -0.028270         NaN         NaN   
2021-07-13         NaN         NaN   -0.012731         NaN         NaN   

                                                                        ...  \
stock      000007.XSHE 000008.XSHE 000009.XSHE 000010.XSHE 000011.XSHE  ...   
dates                                                                   ...   
2014-07-03         NaN         NaN         NaN   -0.039944         NaN  ...   
2014-07-04         NaN         NaN         NaN   -0.017831         NaN  ...   
2014-07-07         NaN         NaN         NaN    0.039329         NaN  ...   
2014-07-08         NaN         NaN         NaN   -0.004364         NaN  ...   
2014-07-09         NaN         NaN         NaN   -0.011696         NaN  ...   
...                ...         ...         ...         ...         ...  ...   
2021-07-07    0.061967         NaN    0.072635   -0.010179   -0.031798  ...   
2021-07-08   -0.054321         NaN    0.108925    0.005130    0.022478  ...   
2021-07-09   -0.053197         NaN    0.017700    0.000000    0.014357  ...   
2021-07-12   -0.049442         NaN   -0.000867   -0.002540   -0.010837  ...   
2021-07-13    0.111122         NaN   -0.055703   -0.002570    0.019390  ...   

                                                                        \
stock      688299.XSHG 688300.XSHG 688321.XSHG 688333.XSHG 688357.XSHG   
dates                                                                    
2014-07-03         NaN         NaN         NaN         NaN         NaN   
2014-07-04         NaN         NaN         NaN         NaN         NaN   
2014-07-07         NaN         NaN         NaN         NaN         NaN   
2014-07-08         NaN         NaN         NaN         NaN         NaN   
2014-07-09         NaN         NaN         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2021-07-07         NaN   -0.023642   -0.037619   -0.001632   -0.025180   
2021-07-08         NaN    0.028197    0.020287    0.005792   -0.004983   
2021-07-09         NaN   -0.011034    0.016731    0.011686   -0.005083   
2021-07-12    0.038999    0.003768    0.001908         NaN    0.047920   
2021-07-13   -0.009064   -0.031971   -0.011902         NaN   -0.052135   

                                                                        
stock      688363.XSHG 688366.XSHG 688368.XSHG 688388.XSHG 688389.XSHG  
dates                                                                   
2014-07-03         NaN         NaN         NaN         NaN         NaN  
2014-07-04         NaN         NaN         NaN         NaN         NaN  
2014-07-07         NaN         NaN         NaN         NaN         NaN  
2014-07-08         NaN         NaN         NaN         NaN         NaN  
2014-07-09         NaN         NaN         NaN         NaN         NaN  
...                ...         ...         ...         ...         ...  
2021-07-07   -0.056925   -0.037927    0.038982    0.006174   -0.020222  
2021-07-08    0.029089   -0.023687   -0.004186    0.069101    0.034904  
2021-07-09   -0.004945   -0.02090

In [19]:
table_y.index

Index(['2014-07-03', '2014-07-04', '2014-07-07', '2014-07-08', '2014-07-09',
       '2014-07-10', '2014-07-11', '2014-07-14', '2014-07-15', '2014-07-16',
       ...
       '2021-06-30', '2021-07-01', '2021-07-02', '2021-07-05', '2021-07-06',
       '2021-07-07', '2021-07-08', '2021-07-09', '2021-07-12', '2021-07-13'],
      dtype='object', name='dates', length=1713)

In [21]:
times = sorted(table_y.index.values)
last_10pct = sorted(table_y.index.values)[-int(0.1*len(times))] # Last 10% of series
last_20pct = sorted(table_y.index.values)[-int(0.2*len(times))] # Last 20% of series

In [22]:
quant_train = quant[(quant.dates< last_20pct)]  # Training data are 80% of total data
quant_val = quant[(quant.dates >= last_20pct) & (quant.dates < last_10pct)]
quant_test = quant[(quant.dates >= last_10pct)]

In [34]:
X_train,y_train1,y_train2 = [],[],[]
for stock in set(quant_train.stock):
    temp = quant_train[quant_train.stock ==stock].drop(['stock','dates'],axis =1)
    if len(temp)>=20:
        v = temp.values
        d0, d1 = v.shape
        s0, s1 = v.strides
        w = 20
        a = stride(v, (d0 - (w - 1), w, d1), (s0, s0, s1))
        for row, values in zip(temp.index, a):
            row = pd.DataFrame(values, columns=temp.columns)
        #     print(row)
            X_train.append(row[['n_pct', 'max_pct','close_vs_forty', 'high_vs_20', 'amt_past5', 'tov_60','max700_vs_min100', 'min_12_100', 'time_f_ipo', 'days_btw_max','max_700', 'min_100', 'close_price', 'buy_price', 'zz_max_vs_min_20','zz_max700_min100']])
            y_train1+=list(row['rtn_1'])
            y_train2.append(row['alpha_1'])

In [35]:
X_test,y_test1,y_test2 = [],[],[]
for stock in set(quant_test.stock):
    temp = quant_test[quant_test.stock ==stock].drop(['stock','dates'],axis =1)
    if len(temp)>=20:
        v = temp.values
        d0, d1 = v.shape
        s0, s1 = v.strides
        w = 20
        a = stride(v, (d0 - (w - 1), w, d1), (s0, s0, s1))
        for row, values in zip(temp.index, a):
            row = pd.DataFrame(values, columns=temp.columns)
        #     print(row)
            X_test.append(row[['n_pct', 'max_pct','close_vs_forty', 'high_vs_20', 'amt_past5', 'tov_60','max700_vs_min100', 'min_12_100', 'time_f_ipo', 'days_btw_max','max_700', 'min_100', 'close_price', 'buy_price', 'zz_max_vs_min_20','zz_max700_min100']])
            y_test1.append(row['rtn_1'])
            y_test2.append(row['alpha_1'])

In [36]:
X_val,y_val1,y_val2 = [],[],[]
for stock in set(quant_val.stock):
    temp = quant_val[quant_val.stock ==stock].drop(['stock','dates'],axis =1)
    if len(temp)>=20:
        v = temp.values
        d0, d1 = v.shape
        s0, s1 = v.strides
        w = 20
        a = stride(v, (d0 - (w - 1), w, d1), (s0, s0, s1))
        for row, values in zip(temp.index, a):
            row = pd.DataFrame(values, columns=temp.columns)
        #     print(row)
            X_val.append(row[['n_pct', 'max_pct','close_vs_forty', 'high_vs_20', 'amt_past5', 'tov_60','max700_vs_min100', 'min_12_100', 'time_f_ipo', 'days_btw_max','max_700', 'min_100', 'close_price', 'buy_price', 'zz_max_vs_min_20','zz_max700_min100']])
            y_val1.append(row['rtn_1'])
            y_val2.append(row['alpha_1'])

In [38]:
# X_train, y_train1,y_train2 = np.array(X_train), np.array(y_train1),np.array(y_train2)
# X_val, y_val1,y_val2 = np.array(X_val), np.array(y_val1),np.array(y_val2)
X_test, y_test1,y_test2 = np.array(X_test), np.array(y_test2),np.array(y_test2)

In [39]:
print('Training set shape', X_train.shape, y_train1.shape,y_train2.shape)
print('Validation set shape', X_val.shape, y_val1.shape,y_val2.shape)
print('Testing set shape' ,X_test.shape, y_test1.shape,y_test2.shape) 

Training set shape (2028633, 20, 16) (2028633, 20) (2028633, 20)
Validation set shape (369099, 20, 16) (369099, 20) (369099, 20)
Testing set shape (302404, 20, 16) (302404, 20) (302404, 20)


In [41]:
np.save('X_train',X_train)
np.save('y_train1',y_train1)
np.save('y_train2',y_train2)
np.save('X_val',X_val)
np.save('y_val1',y_val1)
np.save('y_val2',y_val2)
np.save('X_test',X_test)
np.save('y_test1',y_test1)
np.save('y_test2',y_test2)

In [ ]:
array=[]
for stock in set(quant_train.stock):
    temp = quant_train[quant_train.stock ==stock]
    v = temp.values
    d0, d1 = v.shape
    s0, s1 = v.strides
    w = 20
    a = stride(v, (d0 - (w - 1), w, d1), (s0, s0, s1))
    for row, values in zip(temp.index, a):
        row = pd.DataFrame(values, columns=quant.columns)
    #     print(row)
        array.append(row)